**DEEP LEARNING J-COMPONENT**

**EMOTION CLASSIFICATION**

HARINI GOKULRAM NAIDU 19MIA1004

SHIVANI GOKULRAM NAIDU 19MIA1006

HARSHINI K AIYYER 19MIA1050

ANNUGRAHA S. 19MIA1059

**LOADING THE DATA**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd # importing the dataset 
df_train = pd.read_csv('/content/EC_PROJ_DATASET (1) (1).csv', encoding='utf-8')
df_pred = pd.read_csv('/content/messages_test_features_ready_for_WS_2022 (1) (1).csv',encoding='utf-8')


**IMPORTING LIBRARIES**

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_train, test_size=0.2, random_state=25)

In [ ]:
!pip install spacy

In [ ]:
!pip install text_hammer 
import text_hammer as th

In [ ]:
%%time
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

CPU times: user 448 µs, sys: 912 µs, total: 1.36 ms
Wall time: 1.24 ms


**DATA PREPROCESSING**

In [ ]:
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)

In [ ]:
df_cleaned_train = text_preprocessing(train, 'essay')

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

  0%|          | 0/1488 [00:00<?, ?it/s]

**LABEL ENCODING**

In [ ]:
df_cleaned_train['emotion'] = df_cleaned_train.emotion.replace({'joy':0,'anger':1,'disgust':2,'sadness':3,'fear':4,'surprise':5,'neutral':6})



In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(df_cleaned_train.emotion.values)


**TOKENIZATION**

In [ ]:
from keras.preprocessing.text import Tokenizer
num_words=1000
#this means 10000 unique words can be taken 
tokenizer=Tokenizer(num_words,lower=True)
df_total = pd.concat([df_cleaned_train['essay'], test.essay], axis = 0)
tokenizer.fit_on_texts(df_total)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train=tokenizer.texts_to_sequences(df_cleaned_train['essay']) # this converts texts into some numeric sequences 
X_train_pad=pad_sequences(X_train,maxlen=300,padding='post') # this makes the length of all numeric sequences equal 
X_test = tokenizer.texts_to_sequences(test.essay)
X_test_pad = pad_sequences(X_test, maxlen = 300, padding = 'post')

In [ ]:
!pip install gensim
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

In [ ]:
vector_size = 100
gensim_weight_matrix = np.zeros((num_words ,vector_size))
gensim_weight_matrix.shape

(1000, 100)

In [ ]:
for word, index in tokenizer.word_index.items():
    if index < num_words: # since index starts with zero 
        if word in glove_gensim.wv.vocab:
            gensim_weight_matrix[index] = glove_gensim[word]
        else:
            gensim_weight_matrix[index] = np.zeros(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


**Bi-LSTM model**

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout

In [ ]:
EMBEDDING_DIM = 100
class_num = 7
model = Sequential()
model.add(Embedding(input_dim = num_words,
 output_dim = EMBEDDING_DIM,
 input_length= X_train_pad.shape[1],
 weights = [gensim_weight_matrix],trainable = False))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model.add(Dense(class_num, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam',metrics = 'accuracy')

In [ ]:
history_embedding = model.fit(X_train_pad, y_train, 
                                epochs = 50, batch_size = 128, 
                                verbose = 1, validation_split=0.3)

Epoch 1/50
9/9 [==============================] - 2s 207ms/step - loss: 0.0784 - accuracy: 0.9760 - val_loss: 3.1163 - val_accuracy: 0.3893
Epoch 2/50
9/9 [==============================] - 2s 199ms/step - loss: 0.0737 - accuracy: 0.9769 - val_loss: 3.1767 - val_accuracy: 0.3736
Epoch 3/50
9/9 [==============================] - 2s 220ms/step - loss: 0.0632 - accuracy: 0.9827 - val_loss: 3.1310 - val_accuracy: 0.3937
Epoch 4/50
9/9 [==============================] - 2s 235ms/step - loss: 0.0471 - accuracy: 0.9875 - val_loss: 3.2063 - val_accuracy: 0.3803
Epoch 5/50
9/9 [==============================] - 2s 238ms/step - loss: 0.0617 - accuracy: 0.9827 - val_loss: 3.1434 - val_accuracy: 0.3781
Epoch 6/50
9/9 [==============================] - 2s 236ms/step - loss: 0.0546 - accuracy: 0.9846 - val_loss: 3.1944 - val_accuracy: 0.3982
Epoch 7/50
9/9 [==============================] - 3s 306ms/step - loss: 0.1837 - accuracy: 0.9452 - val_loss: 3.2052 - val_accuracy: 0.3691
Epoch 8/50
9/9 [====